In [1]:
import numpy as np #library to handle data in a vectorized manner
import pandas as pd #library for data analsysis
import requests #library to handle requests
from pandas.io.json import json_normalize #tranform JSON file into a pandas dataframe

from bs4 import BeautifulSoup

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
response = requests.get(wiki_url)
soup = BeautifulSoup(response.content, 'lxml')
Toronto_table = soup.find_all('table')[0]
df = pd.read_html(str(Toronto_table))

In [4]:
neighborhood = pd.DataFrame(df[0])
neighborhood.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
neighborhood.shape

(180, 3)

In [6]:
new_neighborhood = neighborhood[neighborhood.Borough != 'Not assigned'] #change the df by removing rows with borough = Not assigned
new_neighborhood = new_neighborhood.reset_index(drop = True) #reseting the index and finalizing the df with the change
new_neighborhood.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
new_neighborhood.shape

(103, 3)

In [8]:
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [9]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [10]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
merged_df = pd.merge(geo_df, new_neighborhood, on = 'Postal Code')
merged_df = geo_merged[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
